In [1]:
import os
import sys

import openai
from dotenv import load_dotenv, find_dotenv

sys.path.append('../..')

# read local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']

## ChatPromptTemplate包装器

In [6]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = """
You are an expert data scientist
with an expertise in building deep learning models.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Explain the concept of {concept} in a couple of lines"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt = chat_prompt.format_prompt(concept="NLP")
# print(chat_prompt.to_string())
print(chat_prompt.to_messages())

[SystemMessage(content='\nYou are an expert data scientist\nwith an expertise in building deep learning models.\n'), HumanMessage(content='Explain the concept of NLP in a couple of lines')]


## FewShotPromptTemplate包装器

In [7]:
examples = [
    {"input": "高", "output": "矮"},
    {"input": "胖", "output": "瘦"},
    {"input": "精力充沛", "output": "萎靡不振"},
    {"input": "快乐", "output": "伤心"},
    {"input": "黑", "output": "白"},
]

In [8]:
example_prompt = PromptTemplate(input_variables=["input", "output"],
                                template="""
                                词语：{input} \n
                                反义词：{output} \n
                                """)

In [9]:
example_prompt.format(**examples[0])

'\n                                词语：高 \n\n                                反义词：矮 \n\n                                '

In [10]:
from langchain_core.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    example_separator="\n",
    prefix="来玩个反义词接龙游戏，我说词语，你说它的反义词\n",
    suffix="现在轮到你了，词语：{input}\n反义词：",
    input_variables=["input"],
)
few_shot_prompt.format(input="好")

'来玩个反义词接龙游戏，我说词语，你说它的反义词\n\n\n                                词语：高 \n\n                                反义词：矮 \n\n                                \n\n                                词语：胖 \n\n                                反义词：瘦 \n\n                                \n\n                                词语：精力充沛 \n\n                                反义词：萎靡不振 \n\n                                \n\n                                词语：快乐 \n\n                                反义词：伤心 \n\n                                \n\n                                词语：黑 \n\n                                反义词：白 \n\n                                \n现在轮到你了，词语：好\n反义词：'

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=few_shot_prompt)
chain.run("冷")

'冷\n反义词：热'

## CommaSeparatedListOutputParser 输出解释器

In [29]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [30]:
output_parser = CommaSeparatedListOutputParser()

In [31]:
format_instructions = output_parser.get_format_instructions()

In [32]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

In [33]:
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatOpenAI(temperature=0, model_name="gpt-4"), prompt=prompt)

In [37]:
output = chain("ice cream flavors")
output

{'subject': 'ice cream flavors',
 'text': 'Chocolate, Vanilla, Strawberry, Mint Chocolate Chip, Rocky Road'}

In [38]:
output_parser.parse(output['text'])

['Chocolate', 'Vanilla', 'Strawberry', 'Mint Chocolate Chip', 'Rocky Road']

## PydanticOutputParser 输出器

In [6]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, Field, field_validator

In [7]:
model = ChatOpenAI(temperature=0, streaming=True)

In [9]:
# refer: https://docs.pydantic.dev/2.5/errors/usage_errors/#validator-instance-method
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    @field_validator('setup')
    @classmethod
    def question_ends_with_question_mark(cls, field) -> str:
        if field[-1] != "?":
            raise ValueError("question must end with a question mark!")
        return field


joke_query = "Tell me a joke."
output_parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=joke_query)
print(_input.to_messages())
output = model(_input.to_messages())
print(output)
print(type(output))

# AttributeError: 'AIMessage' object has no attribute 'strip'
# print(output_parser.parse(output))

[HumanMessage(content='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"]}\n```\nTell me a joke.\n')]
content='{\n  "setup": "Why did the tomato turn red?",\n  "punchline": "Because it saw the salad dressing!"\n}'


AttributeError: 'AIMessage' object has no attribute 'strip'